In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv('final_card_data.csv')

In [42]:
df.iloc[0]

Card Name                                        Cashback SBI Credit Card
Pros                    Decent 5% cashback on online and 1% cashback o...
Cons                    There are several categories excluded from 1% ...
Card Page Link          https://cardinsider.com/sbi-card/cashback-sbi-...
Apply Link                         https://cardinsider.com/go/sbicashback
Image URL               data:image/svg+xml,%3Csvg%20xmlns='http://www....
Annual Fee                                           ['Shopping', 'Food']
Best For                                              Reward TypeCashback
USP                                                                   NaN
Joining Fees                                                   ₹999 + GST
Renewal Fees                                                   ₹999 + GST
Rewards and Benefits    {'Movie & Dining': 'N/A', 'Rewards Rate': '5% ...
Name: 0, dtype: object

In [43]:
df['Rewards and Benefits'][1]

"{'Movie & Dining': '10% Cashback on Swiggy app including food orders, Instamart, Dineout, and more', 'Rewards Rate': '10% Cashback across Swiggy app, 5% Cashback on online spends, 1% Cashback on other categories', 'Reward Redemption': 'Cashback Against Credit Card Statement', 'Travel': 'Upto 5% cashback on merchants like Uber and Ola', 'Domestic Lounge Access': 'NA', 'International Lounge Access': 'NA', 'Golf': '12 Free Golf Lessons, 4 Free Rounds of Green Fees, and 50% Off on Golf Services', 'Insurance Benefits': 'NA'}"

In [44]:
import ast

# Convert string to dictionary
df['parsed_rewards'] = df['Rewards and Benefits'].apply(lambda x: ast.literal_eval(x))

In [45]:
df['parsed_rewards'][0]

{'Movie & Dining': 'N/A',
 'Rewards Rate': '5% Cashback for Online Spends. 1% Cashback for Offline Spends. Maximum Cashback of ₹5,000 in a Statement Cycle',
 'Reward Redemption': 'Cashback earned with the card will be automatically credited to next month’s statement.',
 'Travel': 'N/A',
 'Domestic Lounge Access': 'N/A',
 'International Lounge Access': 'N/A',
 'Golf': 'N/A',
 'Insurance Benefits': 'N/A'}

In [46]:
# Apply transformation to each row
df['parsed_rewards'] = df['parsed_rewards'].apply(
    lambda reward_dict: {k: v for k, v in reward_dict.items() if v != 'N/A'}
)

In [47]:
df['parsed_rewards'][0]

{'Rewards Rate': '5% Cashback for Online Spends. 1% Cashback for Offline Spends. Maximum Cashback of ₹5,000 in a Statement Cycle',
 'Reward Redemption': 'Cashback earned with the card will be automatically credited to next month’s statement.'}

In [48]:
def rewards_to_text(rewards_dict):
    return " ".join([f"{k}: {v}" for k, v in rewards_dict.items()])

df['benefits_text'] = df['parsed_rewards'].apply(rewards_to_text)

In [49]:
df['benefits_text'][1]

'Movie & Dining: 10% Cashback on Swiggy app including food orders, Instamart, Dineout, and more Rewards Rate: 10% Cashback across Swiggy app, 5% Cashback on online spends, 1% Cashback on other categories Reward Redemption: Cashback Against Credit Card Statement Travel: Upto 5% cashback on merchants like Uber and Ola Domestic Lounge Access: NA International Lounge Access: NA Golf: 12 Free Golf Lessons, 4 Free Rounds of Green Fees, and 50% Off on Golf Services Insurance Benefits: NA'

In [50]:
df

,Card Name,Pros,Cons,Card Page Link,Apply Link,Image URL,Annual Fee,Best For,USP,Joining Fees,Renewal Fees,Rewards and Benefits,parsed_rewards,benefits_text
0,Cashback SBI Credit Card,Decent 5% cashback on online and 1% cashback o...,There are several categories excluded from 1% ...,https://cardinsider.com/sbi-card/cashback-sbi-...,https://cardinsider.com/go/sbicashback,"data:image/svg+xml,%3Csvg%20xmlns='http://www....","['Shopping', 'Food']",Reward TypeCashback,NaN,₹999 + GST,₹999 + GST,"{'Movie & Dining': 'N/A', 'Rewards Rate': '5% ...",{'Rewards Rate': '5% Cashback for Online Spend...,Rewards Rate: 5% Cashback for Online Spends. 1...
1,Swiggy HDFC Bank Credit Card,Get 10% cashback on all your Swiggy food deliv...,Although this card offers great benefits on fo...,https://cardinsider.com/hdfc-bank/swiggy-hdfc-...,https://cardinsider.com/go/hdfcswiggy,"data:image/svg+xml,%3Csvg%20xmlns='http://www....","['Dining', 'Food']",Reward TypeCashback,Complimentary 3-Month Swiggy One Membership,₹500 + GST,₹500 + GST,{'Movie & Dining': '10% Cashback on Swiggy app...,{'Movie & Dining': '10% Cashback on Swiggy app...,Movie & Dining: 10% Cashback on Swiggy app inc...
2,HDFC Bank Millennia Credit Card,Get high 5% cashback on partner merchants like...,You earn a low 1% cash back on offline/online ...,https://cardinsider.com/hdfc-bank/hdfc-bank-mi...,https://cardinsider.com/go/hdfcbank,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",['Shopping'],Reward TypeCashback,"1,000 CashPoints","₹1,000 + GST","₹1,000 + GST",{'Movie & Dining': 'Flat ₹200 Discount on Swig...,{'Movie & Dining': 'Flat ₹200 Discount on Swig...,Movie & Dining: Flat ₹200 Discount on Swiggy D...
3,Amazon Pay ICICI Bank Credit Card,This card is most useful to Amazon loyalist wh...,This card is mainly for shopping with Amazon i...,https://cardinsider.com/icici-bank/amazon-pay-...,https://cardinsider.com/go/iciciamazon,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",['Shopping'],Reward TypeCashback,"Cashback and Discounts Up to ₹2,500 (Both for ...",Lifetime Free,Lifetime Free,{'Movie & Dining': 'A minimum of 15% discount ...,{'Movie & Dining': 'A minimum of 15% discount ...,Movie & Dining: A minimum of 15% discount on d...
4,Flipkart Axis Bank Credit Card,Flipkart Axis Bank Credit Card offers a very g...,Flipkart Axis Bank Credit Card is definitely o...,https://cardinsider.com/axis-bank/flipkart-axi...,https://cardinsider.com/go/axisbank,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",['Shopping'],Reward TypeCashback,Introductory Benefits Worth Rs. 600 on Card Is...,Rs. 500 + GST,Rs. 500 + GST,{'Movie & Dining': '15% Up to ₹500 Off at Part...,{'Movie & Dining': '15% Up to ₹500 Off at Part...,Movie & Dining: 15% Up to ₹500 Off at Partner ...
5,American Express Membership Rewards® Credit Card,Membership Reward Points do not expire. There ...,This card is exclusively available to resident...,https://cardinsider.com/american-express/ameri...,NaN,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",['Shopping'],Reward TypeReward Points,"4,000 Bonus Membership RPs can be availed on S...","₹1,000 + GST","₹4,500 + GST",{'Movie & Dining': 'Get special dining discoun...,{'Movie & Dining': 'Get special dining discoun...,Movie & Dining: Get special dining discounts u...
6,IDFC First Bank Ashva Metal Credit Card,From domestic airports to international ones a...,"To fully take advantage of this card, you woul...",https://cardinsider.com/idfc-first-bank/ashva-...,https://cardinsider.com/go/idfc-ashva,"data:image/svg+xml,%3Csvg%20xmlns='http://www....","['Travel', 'Shopping']",Reward TypeReward Points,"₹2,000 Cashback on Making Four Transactions of...","₹2,999 + GST","₹2,999 + GST",{'Movie & Dining': 'BOGO on Movie Ticket Booki...,{'Movie & Dining': 'BOGO on Movie Ticket Booki...,Movie & Dining: BOGO on Movie Ticket Bookings ...
7,Tata Neu Infinity HDFC Bank Credit Card,10% cashback on the Tata Neu app. Low foreign ...,NeuCoins expire after a period of one year. Th...,https://cardins

In [51]:
import re

def clean_joining_fee(fee):
    if pd.isna(fee):
        return 0
    fee = fee.lower().strip()
    
    # Treat 'nil' or 'lifetime free' as 0
    if 'nil' in fee or 'free' in fee:
        return 0
    
    # Extract numeric part
    match = re.search(r'(\d+[,\d]*)', fee)
    if match:
        num_str = match.group(1).replace(',', '')
        return int(num_str)
    return 0  # fallback if nothing is matched

# Apply to your DataFrame
df["Joining Fees Cleaned"] = df["Joining Fees"].apply(clean_joining_fee)

In [52]:
df["Joining Fees Cleaned"] 

0       999
1       500
2      1000
3         0
4       500
5      1000
6      2999
7      1499
8       500
9         0
10        0
11        0
12     5000
13        0
14     1499
15     5000
16    12500
17     2499
18      999
19      999
Name: Joining Fees Cleaned, dtype: int64

In [53]:
def clean_renewal_fee(fee):
    if pd.isna(fee):
        return 0
    fee = fee.lower().strip()
    
    # Handle free cases
    if 'nil' in fee or 'free' in fee:
        return 0

    # Extract numeric part
    match = re.search(r'(\d+[,\d]*)', fee)
    if match:
        num_str = match.group(1).replace(',', '')
        return int(num_str)
    return 0

# Apply to your DataFrame
df["Renewal Fees Cleaned"] = df["Renewal Fees"].apply(clean_renewal_fee)

In [54]:
df["Renewal Fees Cleaned"]

0       999
1       500
2      1000
3         0
4       500
5      4500
6      2999
7      1499
8       500
9         0
10        0
11        0
12     5000
13        0
14     1499
15     5000
16    12500
17     2499
18      999
19      999
Name: Renewal Fees Cleaned, dtype: int64

In [55]:
df['USP'] = df['USP'].fillna('')

In [56]:
df['combined_features'] = df['USP'] + ' ' + df['benefits_text'] 

In [57]:
df['combined_features'][0]

' Rewards Rate: 5% Cashback for Online Spends. 1% Cashback for Offline Spends. Maximum Cashback of ₹5,000 in a Statement Cycle Reward Redemption: Cashback earned with the card will be automatically credited to next month’s statement.'

In [58]:
df['combined_features_list'] = df['combined_features'].apply(
    lambda x: [sentence.strip() for sentence in x.split('.') if sentence.strip()]
)

In [59]:
df['combined_features_list'][1]

['Complimentary 3-Month Swiggy One Membership Movie & Dining: 10% Cashback on Swiggy app including food orders, Instamart, Dineout, and more Rewards Rate: 10% Cashback across Swiggy app, 5% Cashback on online spends, 1% Cashback on other categories Reward Redemption: Cashback Against Credit Card Statement Travel: Upto 5% cashback on merchants like Uber and Ola Domestic Lounge Access: NA International Lounge Access: NA Golf: 12 Free Golf Lessons, 4 Free Rounds of Green Fees, and 50% Off on Golf Services Insurance Benefits: NA']

In [60]:
import ast

df['Annual Fee'] = df['Annual Fee'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [61]:
df['main_features'] = df['Annual Fee'] + df['combined_features_list']

In [62]:
df['main_features'][1]

['Dining',
 'Food',
 'Complimentary 3-Month Swiggy One Membership Movie & Dining: 10% Cashback on Swiggy app including food orders, Instamart, Dineout, and more Rewards Rate: 10% Cashback across Swiggy app, 5% Cashback on online spends, 1% Cashback on other categories Reward Redemption: Cashback Against Credit Card Statement Travel: Upto 5% cashback on merchants like Uber and Ola Domestic Lounge Access: NA International Lounge Access: NA Golf: 12 Free Golf Lessons, 4 Free Rounds of Green Fees, and 50% Off on Golf Services Insurance Benefits: NA']

In [63]:
!pip install spacy
!python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------- ------------------------------ 2.9/12.8 MB 15.2 MB/s eta 0:00:01
     -------------------------- ------------- 8.4/12.8 MB 20.8 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 21.3 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 19.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
!pip install --upgrade numpy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
!pip uninstall -y h5py
!pip install h5py

Found existing installation: h5py 3.14.0
Uninstalling h5py-3.14.0:
  Successfully uninstalled h5py-3.14.0


You can safely remove it manually.


  Using cached h5py-3.14.0-cp311-cp311-win_amd64.whl.metadata (2.7 kB)
Using cached h5py-3.14.0-cp311-cp311-win_amd64.whl (2.9 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.3.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [67]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

def extract_keywords_spacy(feature_list):
    keywords = set()
    for item in feature_list:
        if isinstance(item, str):
            doc = nlp(item)
            # Extract noun chunks
            for chunk in doc.noun_chunks:
                text = chunk.text.strip().lower()
                if len(text) > 2 and not any(token.is_stop for token in chunk):
                    keywords.add(text.title())
            # Extract named entities (like Swiggy, Uber)
            for ent in doc.ents:
                if ent.label_ in ['ORG', 'PRODUCT', 'GPE']:
                    keywords.add(ent.text.strip().title())
            # Extract single important words (nouns/adjectives)
            for token in doc:
                if token.pos_ in ["NOUN", "PROPN", "ADJ"] and not token.is_stop:
                    keywords.add(token.lemma_.strip().title())
    return list(keywords)

In [68]:
df['refined_features'] = df['main_features'].apply(extract_keywords_spacy)

In [69]:
df['refined_features'][1]

['1% Cashback',
 'Cashback',
 'Complimentary',
 'Merchant',
 'Instamart',
 'Ola',
 'Upto 5% Cashback',
 'Upto',
 'Reward',
 'Spend',
 'Statement',
 'Redemption',
 'Uber And Ola Domestic Lounge Access',
 'Domestic',
 'Credit Card Statement Travel',
 'Month',
 'International',
 'Na International Lounge Access',
 'Green Fees',
 'Movie',
 'Online',
 'Category',
 'Lounge',
 'Free',
 'Swiggy App',
 'Insurance',
 '12 Free Golf Lessons',
 'Swiggy One Membership Movie & Dining',
 '%',
 'Membership',
 'Food',
 '4 Free Rounds',
 'Lessons',
 'Ola Domestic Lounge Access',
 'Card',
 '5% Cashback',
 'Services',
 'Online Spends',
 'Credit',
 'Rounds',
 'Merchants',
 'Benefit',
 'Rate',
 'Access',
 'Na',
 '10% Cashback',
 'Golf',
 'Dining',
 'App',
 'Golf Services Insurance',
 'Travel',
 'Green',
 'Swiggy',
 'Uber',
 'Food Orders',
 'Order',
 'Na Golf',
 'Dineout',
 'Fees',
 'Golf Services Insurance Benefits']

In [70]:
df['keywords'] = df['refined_features'].apply(lambda x: ' '.join(x))

In [71]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['keywords'])

In [72]:
def recommend_cards(user_input, top_n=5):
    """
    Recommends top N credit cards based on user input preferences.

    Parameters:
    - user_input: dict with keys:
        'max_joining_fee' (int),
        'max_renewal_fee' (int),
        'reward_preferences' (str),
        'welcome_bonus_preference' (bool)

    Returns:
    - DataFrame of top recommended cards with details
    """
    
    filtered_df = df.copy()
    if 'max_joining_fee' in user_input:
        filtered_df = filtered_df[
            pd.to_numeric(filtered_df['Joining Fees Cleaned'], errors='coerce').fillna(0) <= user_input['max_joining_fee']
        ]
    if 'max_renewal_fee' in user_input:
        filtered_df = filtered_df[
            pd.to_numeric(filtered_df['Renewal Fees Cleaned'], errors='coerce').fillna(0) <= user_input['max_renewal_fee']
        ]

    if filtered_df.empty:
        return "No cards match your fee preferences."

    
    user_query = user_input.get('reward_preferences', '')
    user_vector = vectorizer.transform([user_query])

    
    similarity_scores = cosine_similarity(user_vector, tfidf_matrix[filtered_df.index]).flatten()

    
    if user_input.get('welcome_bonus_preference', False):
        bonus_boost = filtered_df['USP'].apply(lambda x: 0.1 if isinstance(x, str) and len(x) > 10 else 0)
        similarity_scores += bonus_boost.values

    
    top_indices = similarity_scores.argsort()[::-1][:top_n]
    recommendations = filtered_df.iloc[top_indices].copy()
    recommendations['Similarity Score'] = similarity_scores[top_indices]

    
    return recommendations[
        ['Image URL','Card Name', 'Joining Fees', 'Renewal Fees', 'USP', 'Card Page Link', 'combined_features','Pros','Cons' ,'Similarity Score']
    ]

In [73]:
user_input = {
    'max_joining_fee': 5000,
    'max_renewal_fee': 999,
    'reward_preferences': 'travel  Zomato amazon Personal Accident Cover',
    'welcome_bonus_preference': True
}

In [74]:
df['combined_features'][6]

'₹2,000 Cashback on Making Four Transactions of ₹500 or Above Movie & Dining: BOGO on Movie Ticket Bookings Upto ₹400 With District by Zomato, Twice Each Month (Annual Benefit - ₹9,600) Rewards Rate: 10X Reward Points/₹150 on Incremental Spends Above ₹20,000 Per Statement Cycle. 5X Reward Points/₹150 on Spends Upto ₹20,000. 3X RPs Are Also Awarded for Rent, Wallet Load, Education, and Government Transactions Reward Redemption: Redeem for Vouchers/Cash Against Statement or Pay With Points. 1 RP = ₹0.25 Travel: 4 Complimentary Railway Lounge Access Each Quarter, 16 Each Year (For Both Primary and Add-On Cardholders) Domestic Lounge Access: 4 Complimentary Lounge Access Each Quarter International Lounge Access: 2 Complimentary Lounge Access Each Quarter Golf: 2 Free Golf Rounds/Lessons on Spends of ₹20,000 Each in a Statement Cycle Insurance Benefits: Purchase Protection, Personal Accident Cover, Free Trip Cancellation Protection, Travel Insurance and Credit Shield'

In [75]:
df['combined_features'][4]

'Introductory Benefits Worth Rs. 600 on Card Issuance Movie & Dining: 15% Up to ₹500 Off at Partner Restaurants With Axis Bank Dining Delights Program Rewards Rate: 5% Cashback on Flipkart & Cleartrip, 4% Cashback on Preferred Partners (Swiggy, Uber, PVR, and cult.fit), and 1% Cashback on All Other Spends Travel: Accelerated Cashback (4%) on Uber Rides'

In [76]:
recommend_cards(user_input, top_n=5)

,Image URL,Card Name,Joining Fees,Renewal Fees,USP,Card Page Link,combined_features,Pros,Cons,Similarity Score
8,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",Airtel Axis Bank Credit Card,₹500 + GST,₹500 + GST,Amazon Voucher Worth Rs. 500,https://cardinsider.com/axis-bank/airtel-axis-...,Amazon Voucher Worth Rs. 500 Movie & Dining: 1...,You would get 25% exclusive cashback on Airtel...,"With the Airtel Axis Bank Credit Card, you hav...",0.236189
3,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",Amazon Pay ICICI Bank Credit Card,Lifetime Free,Lifetime Free,"Cashback and Discounts Up to ₹2,500 (Both for ...",https://cardinsider.com/icici-bank/amazon-pay-...,"Cashback and Discounts Up to ₹2,500 (Both for ...",This card is most useful to Amazon loyalist wh...,This card is mainly for shopping with Amazon i...,0.215071
13,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",IDFC FIRST Select Credit Card,Nil,Nil,"Vouchers worth ₹500 on spends of ₹5,000 within...",https://cardinsider.com/idfc-first-bank/idfc-f...,"Vouchers worth ₹500 on spends of ₹5,000 within...",This is a lifetime free credit card without an...,There isn’t any complimentary international lo...,0.213856
9,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",Axis Bank Neo Credit Card,Nil,Nil,100% Cashback Up to ₹300 on First Utility Bill...,https://cardinsider.com/axis-bank/axis-bank-ne...,100% Cashback Up to ₹300 on First Utility Bill...,Get 10% off while booking movies with BookMySh...,There are no insurance benefits provided with ...,0.161941
18,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",MakeMyTrip ICICI Bank Credit Card,₹999 + GST,₹999 + GST,"₹1,000 MMT Voucher and MMTBLACK Gold Tier Memb...",https://cardinsider.com/icici-bank/makemytrip-...,"₹1,000 MMT Voucher and MMTBLACK Gold Tier Memb...",NaN,NaN,0.125243


In [77]:
import pickle

In [79]:
pickle.dump(df,open('df_new.pkl','wb'))

In [80]:
df['Image URL']

0     data:image/svg+xml,%3Csvg%20xmlns='http://www....
1     data:image/svg+xml,%3Csvg%20xmlns='http://www....
2     data:image/svg+xml,%3Csvg%20xmlns='http://www....
3     data:image/svg+xml,%3Csvg%20xmlns='http://www....
4     data:image/svg+xml,%3Csvg%20xmlns='http://www....
5     data:image/svg+xml,%3Csvg%20xmlns='http://www....
6     data:image/svg+xml,%3Csvg%20xmlns='http://www....
7     data:image/svg+xml,%3Csvg%20xmlns='http://www....
8     data:image/svg+xml,%3Csvg%20xmlns='http://www....
9     data:image/svg+xml,%3Csvg%20xmlns='http://www....
10    data:image/svg+xml,%3Csvg%20xmlns='http://www....
11    data:image/svg+xml,%3Csvg%20xmlns='http://www....
12    data:image/svg+xml,%3Csvg%20xmlns='http://www....
13    data:image/svg+xml,%3Csvg%20xmlns='http://www....
14    data:image/svg+xml,%3Csvg%20xmlns='http://www....
15    data:image/svg+xml,%3Csvg%20xmlns='http://www....
16    data:image/svg+xml,%3Csvg%20xmlns='http://www....
17    data:image/svg+xml,%3Csvg%20xmlns='http://